In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [ ]:
train_path = '../input/credit-dset/clean_trained_outlier.csv'
test_path = '../input/credit-dset/test_cleaned_outlier.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

if (test_path == '../input/credit-dset/test_cleaned.csv'):
    test_df.drop(['Number'], axis=1, inplace=True)

In [ ]:
categorical_cols = []
numerical_cols = []

for col in train_df.columns:
    if col != 'Credit_Score':
        if train_df[col].dtype == 'object':
            categorical_cols.append(col)
        else:
            numerical_cols.append(col)
    else:
        print('Skipping Credit_Score column')

print(categorical_cols)
print(numerical_cols)

In [ ]:
# string -> no of months
def convert_to_2_cols(s):
    parts = s.split('_spent_')
    spent = parts[0]
    payment = parts[1].split('_payments')[0]
    return pd.Series([spent,payment])

train_df[['Spent', 'Value_Payments']] = train_df['Payment_Behaviour'].apply(convert_to_2_cols)
train_df.drop(['Payment_Behaviour'], axis=1, inplace=True)

test_df[['Spent', 'Value_Payments']] = test_df['Payment_Behaviour'].apply(convert_to_2_cols)
test_df.drop(['Payment_Behaviour'], axis=1, inplace=True)

In [ ]:
encoder = OneHotEncoder(sparse_output=False)

month_encoded = encoder.fit_transform(train_df[['Month']])
month_df = pd.DataFrame(month_encoded, columns=encoder.get_feature_names_out(['Month']))

encoded_columns = encoder.fit_transform(train_df[['Profession', 'Payment_of_Min_Amount']])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['Profession', 'Payment_of_Min_Amount']))
train_df = pd.concat([train_df, encoded_df, month_df], axis=1)

train_df.drop(['Month', 'Profession', 'Payment_of_Min_Amount'], axis=1, inplace=True)

label_encoder = LabelEncoder()
train_df['Credit_Mix'] = label_encoder.fit_transform(train_df[
    'Credit_Mix'
])

train_df['Spent'] = label_encoder.fit_transform(train_df[
    'Spent'
])

train_df['Value_Payments'] = label_encoder.fit_transform(train_df[
    'Value_Payments'
])